In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import csv

In [7]:
# Setup Chrome
service = Service(r"E:\adobe\CDE\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get("https://www.tradingview.com/markets/stocks-usa/market-movers-all-stocks/")
wait = WebDriverWait(driver, 20)
actions = ActionChains(driver)

In [8]:
# Accept cookies if present
try:
    accept_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(),"Accept")]')))
    accept_btn.click()
except:
    pass

# Scroll/Click loop
last_row_count = 0
same_count = 0
max_same = 5

while True:
    try:
        # Scroll down slightly to trigger lazy loading
        driver.execute_script("window.scrollBy(0, 500);")
        time.sleep(1)

        # Scroll back to top (so "Load More" is visible)
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)

        # Click Load More if visible
        load_more_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "content-D4RPB3ZC")))
        actions.move_to_element(load_more_btn).perform()
        load_more_btn.click()
        time.sleep(2)

        # Track row count
        rows = driver.find_elements(By.TAG_NAME, 'tr')
        print(f"✅ Loaded rows: {len(rows)}")

        if len(rows) == last_row_count:
            same_count += 1
        else:
            same_count = 0

        if same_count >= max_same or len(rows) >= 4576:
            break

        last_row_count = len(rows)

    except Exception as e:
        print(f"⚠️ Error: {e}")
        break
# After scrolling, grab all rows
rows = driver.find_elements(By.CSS_SELECTOR, 'tr.row-RdUXZpkv.listRow')
symbols = []

for row in rows:
    data_key = row.get_attribute('data-rowkey')
    if data_key and ':' in data_key:
        symbols.append(data_key)

print(f"Total symbols collected: {len(symbols)}")
 

✅ Loaded rows: 202
✅ Loaded rows: 302
✅ Loaded rows: 402
✅ Loaded rows: 502
✅ Loaded rows: 602
✅ Loaded rows: 702
✅ Loaded rows: 802
✅ Loaded rows: 902
✅ Loaded rows: 1002
✅ Loaded rows: 1102
✅ Loaded rows: 1202
✅ Loaded rows: 1302
✅ Loaded rows: 1402
✅ Loaded rows: 1502
✅ Loaded rows: 1602
✅ Loaded rows: 1702
✅ Loaded rows: 1802
✅ Loaded rows: 1902
✅ Loaded rows: 2002
✅ Loaded rows: 2102
✅ Loaded rows: 2202
✅ Loaded rows: 2302
✅ Loaded rows: 2402
✅ Loaded rows: 2502
✅ Loaded rows: 2602
✅ Loaded rows: 2702
✅ Loaded rows: 2802
✅ Loaded rows: 2902
✅ Loaded rows: 3002
✅ Loaded rows: 3102
✅ Loaded rows: 3202
✅ Loaded rows: 3302
✅ Loaded rows: 3402
✅ Loaded rows: 3502
✅ Loaded rows: 3602
✅ Loaded rows: 3702
✅ Loaded rows: 3802
✅ Loaded rows: 3902
✅ Loaded rows: 4002
✅ Loaded rows: 4102
✅ Loaded rows: 4202
✅ Loaded rows: 4302
✅ Loaded rows: 4402
✅ Loaded rows: 4502
✅ Loaded rows: 4589
Total symbols collected: 4587


In [9]:
def safe_text(by, value):
    try:
        return driver.find_element(by, value).text.strip()
    except:
        return ""

fields = {
    "name": (By.TAG_NAME, 'h1'),
    "reported_date": (By.CLASS_NAME, 'wrapper-QCJM7wcY'),
    "reported_period": (By.CLASS_NAME, 'container-aTgWtsSD'),
    "eps_estimate": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[3]/div[2]/div/div[3]/div[2]/div[1]'),
    "revenue": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[3]/div[2]/div/div[4]/div[2]/div[1]/div'),
    "market_cap": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[4]/div[2]/div/div[1]/div[2]/div[1]/div'),
    "dividend": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[4]/div[2]/div/div[2]/div[2]/div[1]/div'),
    "earning": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[4]/div[2]/div/div[3]/div[2]/div[1]/div'),
    "basic_eps": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[4]/div[2]/div/div[4]/div[2]/div[1]/div'),
    "net_income": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[4]/div[2]/div/div[5]/div[2]/div[1]/div'),
    "fy_revenue": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[4]/div[2]/div/div[6]/div[2]/div[1]/div'),
    "shares": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[4]/div[2]/div/div[7]/div[2]/div[1]/div'),
    "beta": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[4]/div[2]/div/div[8]/div[2]/div[1]/div'),
    "employees": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[5]/div[2]/div/div[1]/div[2]/div[1]/div'),
    "change": (By.CLASS_NAME, 'changeRed-mgaPdXvL'),
    "emp_revenue": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[5]/div[2]/div/div[3]/div[2]/div[1]/div'),
    "emp_income": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[5]/div[2]/div/div[4]/div[2]/div[1]/div'),
    "sector": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[6]/div[2]/div/div[1]/div[1]/div[2]/div[1]/a/div'),
    "industry": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[6]/div[2]/div/div[1]/div[2]/div[2]/div[1]/a/div'),
    "ceo": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[6]/div[2]/div/div[1]/div[3]/div[2]/div[1]/div'),
    "website": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[6]/div[2]/div/div[1]/div[4]/div[2]/div[1]/a/div'),
    "headquater": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[6]/div[2]/div/div[1]/div[5]/div[2]/div[1]/div'),
    "founded": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[6]/div[2]/div/div[1]/div[6]/div[2]/div[1]/div'),
    "figi": (By.XPATH, '//*[@id="symbol-overview-page-section"]/div/div/div[6]/div[2]/div/div[1]/div[7]/div[2]/div[1]/div'),
}

required_fields = ["name", "reported_date", "eps_estimate", "market_cap", "sector"]

all_data = []

for i, key in enumerate(symbols):
    exchange, symbol = key.split(":")
    url = f"https://www.tradingview.com/symbols/{exchange}-{symbol}/"
    driver.get(url)
    print(f"[{i+1}/{len(symbols)}] Visiting {url}")

    # Initialize data with None for all fields
    data = {"symbol": symbol}
    for field in fields.keys():
        data[field] = None

    try:
        # Wait for page to load (using any element)
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//body')))
        
        # Populate data fields
        for field, (by, selector) in fields.items():
            data[field] = safe_text(by, selector)
        
        print(f"✅ Data collected for {symbol}")
    
    except Exception as e:
        print(f"⚠️ Partial data for {symbol}: {e}")
    
    finally:
        # Always add to results even with errors
        all_data.append(data)
        time.sleep(1)  # Be polite with requests



[1/4587] Visiting https://www.tradingview.com/symbols/NYSE-A/
✅ Data collected for A
[2/4587] Visiting https://www.tradingview.com/symbols/NYSE-AA/
✅ Data collected for AA
[3/4587] Visiting https://www.tradingview.com/symbols/NASDAQ-AACB/
✅ Data collected for AACB
[4/4587] Visiting https://www.tradingview.com/symbols/NYSE-AACT/
✅ Data collected for AACT
[5/4587] Visiting https://www.tradingview.com/symbols/NASDAQ-AAL/
✅ Data collected for AAL
[6/4587] Visiting https://www.tradingview.com/symbols/NYSE-AAM/
✅ Data collected for AAM
[7/4587] Visiting https://www.tradingview.com/symbols/NASDAQ-AAME/
✅ Data collected for AAME
[8/4587] Visiting https://www.tradingview.com/symbols/NYSE-AAMI/
✅ Data collected for AAMI
[9/4587] Visiting https://www.tradingview.com/symbols/NASDAQ-AAOI/
✅ Data collected for AAOI
[10/4587] Visiting https://www.tradingview.com/symbols/NASDAQ-AAON/
✅ Data collected for AAON
[11/4587] Visiting https://www.tradingview.com/symbols/NYSE-AAP/
✅ Data collected for AAP
[12

In [10]:
header = [
    'symbol', 'name', 'reported_date', 'reported_period', 'eps_estimate', 'revenue', 'market_cap', 'dividend',
    'earning', 'basic_eps', 'net_income', 'fy_revenue', 'shares', 'beta', 'employees',
    'change', 'emp_revenue', 'emp_income', 'sector', 'industry', 'ceo', 'website', 'headquater', 'founded', 'figi'
]

# Ensure all entries have all header fields, even if some are missing
for row in all_data:
    for field in header:
        if field not in row:
            row[field] = ""

# Write to CSV
with open("Task3.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=header)
    writer.writeheader()
    writer.writerows(all_data)

print("✅ Done! Data saved to Task3.csv")



✅ Done! Data saved to Task3.csv
